In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/document/Cargill Specs Document.pdf


## Phase 1A

In [2]:
!pip install -q pdfplumber faiss-cpu nltk transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [3]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            content = page.extract_text()
            if content:
                text += content + "\n"
    return text

pdf_path = "/kaggle/input/document/Cargill Specs Document.pdf"
raw_text = extract_text_from_pdf(pdf_path)
print("✅ Extracted Text Sample:\n", raw_text[:1000])

✅ Extracted Text Sample:
 Dairy Enteligen (iOS & Android) Application
---------------------------------------------------------------------
Specifications Document
Version 1.0
Table of Contents
Table of Contents 2
Introduction 10
Character Limits & Other Validations 10
Validation Messages 11
Empty Screen Messages 14
Main Menu & Bottom Navigation 15
User Onboarding 15
User Authentication 17
Cargill Login 17
Action Buttons 17
Login 17
Can’t access your account 17
Login with Okta 18
Okta Login 18
Action Buttons 18
Login 18
Forgot Password 19
Action Buttons 19
Send me the link! 19
Back to Login 19
Reset Password 19
Action Buttons 20
Reset my password! 20
Customer Management 21
Customers Listing 21
Filters 21
View Customer 21
Sites 22
Recent Visits (sorted by latest updated) 22
Edit Customer 22
Action Buttons 23
Update 23
Cancel 23
Sharing of Customers Among DFCs 23
Prospect Management 24
Prospects Listing 24
Filters 24
Create Prospect 24
Action Buttons 29
Save 29
Cancel 29
View Prospect 29

In [4]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

def chunk_text(text, chunk_token_limit=256):
    sentences = sent_tokenize(text)
    chunks, chunk = [], ""
    for sent in sentences:
        if len(tokenizer.encode(chunk + sent)) <= chunk_token_limit:
            chunk += " " + sent
        else:
            chunks.append(chunk.strip())
            chunk = sent
    if chunk:
        chunks.append(chunk.strip())
    return chunks

chunks = chunk_text(raw_text)
print(f"✅ Total Chunks: {len(chunks)}")
print("🧩 Sample Chunk:\n", chunks[0])

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1931 > 512). Running this sequence through the model will result in indexing errors


✅ Total Chunks: 239
🧩 Sample Chunk:
 Dairy Enteligen (iOS & Android) Application
---------------------------------------------------------------------
Specifications Document
Version 1.0
Table of Contents
Table of Contents 2
Introduction 10
Character Limits & Other Validations 10
Validation Messages 11
Empty Screen Messages 14
Main Menu & Bottom Navigation 15
User Onboarding 15
User Authentication 17
Cargill Login 17
Action Buttons 17
Login 17
Can’t access your account 17
Login with Okta 18
Okta Login 18
Action Buttons 18
Login 18
Forgot Password 19
Action Buttons 19
Send me the link! 19
Back to Login 19
Reset Password 19
Action Buttons 20
Reset my password!


In [5]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('all-MiniLM-L6-v2')
chunk_embeddings = embedder.encode(chunks, show_progress_bar=True)


2025-05-05 09:39:39.185779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746437979.392865      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746437979.455048      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
import faiss
import numpy as np

dimension = chunk_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))


In [7]:
from transformers import pipeline

qa_model = pipeline("text2text-generation", model="google/flan-t5-small")

def ask_query_with_context(query, k=3, sim_threshold=1.5):
    query_embedding = embedder.encode([query])
    D, I = index.search(np.array(query_embedding), k)
    
    # Filter based on distance
    if D[0][0] > sim_threshold:
        print("🔴 No confident match found. Use fallback or say 'insufficient context.'")
        return None

    context = " ".join([chunks[i] for i in I[0]])
    
    # Better prompt
    prompt = f"Answer the following question using the context below:\n\nContext:\n{context}\n\nQuestion: {query}"

    result = qa_model(prompt, max_new_tokens=100)
    return result[0]['generated_text']

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
general_queries = [
    "What is the standard shelf life of cooked ground beef patties?",
    "What are the net weight requirements for the product?",
    "How should products be stored and handled according to the specification?",
    "What are the temperature tolerances during distribution and storage?"
]

for q in general_queries:
    print(f"\n🔎 Query: {q}")
    answer = ask_query_with_context(q)
    if answer:
        print("🧠 Answer:", answer)


🔎 Query: What is the standard shelf life of cooked ground beef patties?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


🧠 Answer: 3.

🔎 Query: What are the net weight requirements for the product?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: as-fed / cow

🔎 Query: How should products be stored and handled according to the specification?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: a.

🔎 Query: What are the temperature tolerances during distribution and storage?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
allergen_queries = [
    "What allergens are present in the product?",
    "What is the complete ingredient list for the Angus Beef Patty 80/20?",
    "Is there any soy or gluten in the ingredients?"
]

for q in allergen_queries:
    print(f"\n🔎 Query: {q}")
    answer = ask_query_with_context(q)
    if answer:
        print("🧠 Answer:", answer)


🔎 Query: What allergens are present in the product?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: GEA, Avg.

🔎 Query: What is the complete ingredient list for the Angus Beef Patty 80/20?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: 1.

🔎 Query: Is there any soy or gluten in the ingredients?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


🧠 Answer: Gluten


In [11]:
packaging_queries = [
    "What is the packaging configuration for each master case?",
    "What are the packaging material specifications?",
    "How many patties are packed per layer and per case?"
]

for q in packaging_queries:
    print(f"\n🔎 Query: {q}")
    answer = ask_query_with_context(q)
    if answer:
        print("🧠 Answer:", answer)


🔎 Query: What is the packaging configuration for each master case?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔴 No confident match found. Use fallback or say 'insufficient context.'

🔎 Query: What are the packaging material specifications?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: 1. No

🔎 Query: How many patties are packed per layer and per case?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: 2


In [12]:
labeling_queries = [
    "What labeling instructions are required for retail packaging?",
    "Does the specification mention USDA inspection requirements?",
    "What nutritional labeling elements are included or required?"
]

for q in labeling_queries:
    print(f"\n🔎 Query: {q}")
    answer = ask_query_with_context(q)
    if answer:
        print("🧠 Answer:", answer)


🔎 Query: What labeling instructions are required for retail packaging?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: avg. TMR - Std. See Validation Messages for details (Changing values coming from Site or Pen Setup Milk Number Fetched from Site Setup  Milk Priduction Yield (Kg) Production (Kgs) The validations applied should be the same as Site Setup If blank in site setup, will show no value

🔎 Query: Does the specification mention USDA inspection requirements?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: no

🔎 Query: What nutritional labeling elements are included or required?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: a, b, c, d, e, g, h, i, k, l, m, n, o, p, r, s, t, u, v, w, x, y, z


In [13]:
quality_queries = [
    "What are the tolerances for product thickness and diameter?",
    "What sensory characteristics are required for finished products?",
    "What microbiological standards must the product meet?"
]

for q in quality_queries:
    print(f"\n🔎 Query: {q}")
    answer = ask_query_with_context(q)
    if answer:
        print("🧠 Answer:", answer)


🔎 Query: What are the tolerances for product thickness and diameter?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: 0

🔎 Query: What sensory characteristics are required for finished products?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: X Avg No

🔎 Query: What microbiological standards must the product meet?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: 1.


In [14]:
operations_queries = [
    "What is the target cooking temperature for the final product?",
    "Are there any specific grinding or forming requirements?",
    "What procedures are followed for traceability and recalls?"
]

for q in operations_queries:
    print(f"\n🔎 Query: {q}")
    answer = ask_query_with_context(q)
    if answer:
        print("🧠 Answer:", answer)



🔎 Query: What is the target cooking temperature for the final product?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: a = 0

🔎 Query: Are there any specific grinding or forming requirements?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: 1.

🔎 Query: What procedures are followed for traceability and recalls?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 Answer: a.


## Phase 1B